In [1]:
import os

os.chdir("..")

current_directory = os.getcwd()
print("Current Working Directory:", current_directory)

Current Working Directory: c:\Users\max-s\Desktop\Uni\Master\Masterthesis\Code\git\overparametrized_dnn


In [2]:
from data.data_generator import get_data, preprocess
from src.my_dnn import create_dnn
from sklearn.model_selection import train_test_split
import numpy as np
import keras
import keras_tuner as kt
import tensorflow as tf

In [3]:
def regression_func(x):
    return np.exp(np.linalg.norm(x, axis=1))

input_dim = 7

x, y = get_data(regression_func, x_dim=input_dim, num_samples=100, sigma=0.05)
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2)

train_data = preprocess(x_train, y_train, batch_size=20, training=True)
val_data = preprocess(x_val, y_val, batch_size=20, training=False)

In [4]:
def model_builder(hp):
    hp_units = hp.Int("units", min_value=10, max_value=40, step=10)
    hp_layers = hp.Int("layers", min_value=5, max_value=20, step=5)
    hp_nets = hp.Int("networks", min_value=80, max_value=6 * 80, step=80)
    hp_beta = hp.Float('beta', min_value=20, max_value=80)
    hp_gamma = hp.Float('gamma', min_value=30, max_value=80)
    hp_learning_rate = hp.Float(
        "learning_rate", min_value=np.exp(-16), max_value=np.exp(-14), sampling="log"
    )

    # Hier nutzen Sie Ihre angepasste `create_dnn` Funktion mit den hp-Argumenten
    model = create_dnn(
        train_shape=x_train.shape,  # Beispielwerte
        num_networks=hp_nets,
        num_layers=hp_layers,
        num_neurons=hp_units,
        beta=hp_beta,
        gamma=hp_gamma,
        delta=1,
    )

    optimizer = keras.optimizers.SGD(learning_rate=hp_learning_rate)

    model.compile(
        optimizer=optimizer, loss="mean_squared_error", metrics=["mean_squared_error"]
    )

    return model

In [5]:
tuner = kt.RandomSearch(
    model_builder,
    objective='val_loss',
    max_trials=20,
    executions_per_trial=2,
    directory='my_dir',
    project_name='keras_tuner_2003'
)

# Starten des Tuning-Prozesses
tuner.search(x_train, y_train, epochs=500, validation_data=(x_val, y_val))

# Abrufen der besten Hyperparameter
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]



Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
30                |30                |units
20                |20                |layers
480               |480               |networks
32.189            |32.189            |beta
59.044            |59.044            |gamma
2.9852e-07        |2.9852e-07        |learning_rate

Epoch 1/500



In [9]:
print(f"""
Die besten Hyperparameter sind:
- Anzahl der Netzwerke: {best_hps.get('networks')}
- Anzahl der Schichten: {best_hps.get('layers')}
- Anzahl der Neuronen: {best_hps.get('units')}
- Delta: {best_hps.get('learning_rate')}
""")


Die besten Hyperparameter sind:
- Anzahl der Netzwerke: 160
- Anzahl der Schichten: 10
- Anzahl der Neuronen: 20
- Delta: 4.156935364128902e-07



In [4]:
def train_and_evaluate_dnn(model, train_data, test_data, epochs=75):
    """
    Trains the model on the given data and evaluates its performance.
    """
    model.fit(train_data, epochs=epochs, verbose=0)
    mse, mae = model.evaluate(test_data, verbose=0)
    return mse, mae

In [5]:
mses = []  # Initialize empty list to store MSEs
maes = []  # Initialize empty list to store MAEs
for _ in range(1):
    x_train, y_train = get_data(
        regression_func, x_dim=input_dim, num_samples=100, sigma=0.05
    )
    x_test, y_test = get_data(
        regression_func, x_dim=input_dim, num_samples=10**5, sigma=0.05
    )

    # Preprocess data
    train_data = preprocess(x_train, y_train, batch_size=100, training=True)
    test_data = preprocess(x_test, y_test, batch_size=100, training=False)

    model = create_dnn(
        train_shape=x_train.shape,  # Beispielwerte
        num_networks=160,
        num_layers=10,
        num_neurons=20,
        beta=80,
        gamma=80,
        delta=1,
    )

    optimizer = keras.optimizers.SGD(learning_rate=np.exp(-15))

    model.compile(
        optimizer=optimizer, loss="mean_squared_error", metrics=["mean_squared_error"]
    )
    mse, mae = train_and_evaluate_dnn(model, train_data, test_data, epochs=500)
    mses.append(mse)
    maes.append(mae)

NotImplementedError: in user code:

    File "c:\Users\max-s\Desktop\Uni\Master\Masterthesis\Code\git\overparametrized_dnn\src\my_dnn.py", line 176, in train_step  *
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
    File "c:\Users\max-s\anaconda3\envs\py4ds\Lib\site-packages\keras\src\optimizers\base_optimizer.py", line 269, in apply_gradients  **
        self.apply(grads, trainable_variables)
    File "c:\Users\max-s\anaconda3\envs\py4ds\Lib\site-packages\keras\src\optimizers\base_optimizer.py", line 334, in apply
        variable.assign(variable.constraint(variable))
    File "c:\Users\max-s\Desktop\Uni\Master\Masterthesis\Code\git\overparametrized_dnn\src\my_dnn.py", line 65, in __call__
        return self.apply_l1_projection(w)
    File "c:\Users\max-s\anaconda3\envs\py4ds\Lib\site-packages\tensorflow\core\function\polymorphism\function_type.py", line 583, in canonicalize_to_monomorphic
        _make_validated_mono_param(name, arg, poly_parameter.kind,
   

In [11]:
print(mses)

[0.95123291015625]


In [11]:
x_val, y_val = get_data(m1, x_dim=m1.expected_dim, num_samples=10**5)
validation_data = preprocess(x_val, y_val, training=False)

model.evaluate(validation_data)

1563/1563 [==============================] - 26s 17ms/step - loss: 0.0071 - mean_squared_error: 0.0071


[0.007052791304886341, 0.007052791304886341]

In [15]:
def test_norm_l1(model):
    # Test if L1 projection of last layer worked
    weights = tf.reshape(model.trainable_variables[-1], [-1])
    norm = tf.norm(weights, ord=1)
    print(f"norm: {norm}, gamma: {model.gamma}")


def test_norm_l2(model):
    # Test if L2 projection of inner weights worked
    current_weights = tf.concat(
        [tf.reshape(v, [-1]) for v in model.trainable_weights[:-1]], axis=0
    )
    sub_nets_init_weights = model.sub_nets_init_weights
    diff = sub_nets_init_weights - current_weights
    norm = tf.norm(diff)
    print(f"norm: {norm}, delta: {model.delta}")

In [16]:
test_norm_l1(model)
test_norm_l2(model)

norm: 0.7622666954994202, gamma: 10
norm: 0.0030101335141807795, delta: 1.0
